# Agent Based Model: Main Notebook

In [5162]:
import numpy.random as rnd
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
import random

#### Workflow Notes 
* Draw Students with values within the four dimensions 
* Function that creates a study group (draw 4 students) -> Measure homogeniety 
* Task 


## Importing The Study Groups that are to be Tested

In [5163]:
homogenous = pd.read_csv("Conditions/Homogenous.csv")
fiftyfifty = pd.read_csv("Conditions/FiftyFifty.csv")
heterogenous = pd.read_csv("Conditions/Heterogenous.csv")


for i in (range(len(homogenous.index))):
    print(list(homogenous.iloc[i][1:5]))


['ESFJ', 'ESFJ', 'ESFJ', 'ESFJ']
['ESTJ', 'ESTJ', 'ESTJ', 'ESTJ']
['ISFJ', 'ISFJ', 'ISFJ', 'ISFJ']
['ISTJ', 'ISTJ', 'ISTJ', 'ISTJ']
['ENFP', 'ENFP', 'ENFP', 'ENFP']
['ENFJ', 'ENFJ', 'ENFJ', 'ENFJ']
['INFP', 'INFP', 'INFP', 'INFP']
['INFJ', 'INFJ', 'INFJ', 'INFJ']
['ENTP', 'ENTP', 'ENTP', 'ENTP']
['ENTJ', 'ENTJ', 'ENTJ', 'ENTJ']
['INTP', 'INTP', 'INTP', 'INTP']
['INTJ', 'INTJ', 'INTJ', 'INTJ']
['ESFP', 'ESFP', 'ESFP', 'ESFP']
['ESTP', 'ESTP', 'ESTP', 'ESTP']
['ISFP', 'ISFP', 'ISFP', 'ISFP']
['ISTP', 'ISTP', 'ISTP', 'ISTP']


## Creating Study Groups

#### Practical Functions

In [5164]:
def n_sampler(size):
    '''
    sample a number of random numbers from the beta distribution
    '''
    max_vals = []
    min_vals = []

    beta_dist = rnd.beta(2.5, 2.5, size)

    for i in range(size):
        if beta_dist[i] >= 0.5:
            max_vals.append(beta_dist[i])
        else:
            min_vals.append(beta_dist[i])
    
    return max_vals, min_vals

max_vals, min_vals = n_sampler(1000000)

In [5165]:
def data_collect(studygroup, student_list):
    '''
    function to collect data from the simulation
    '''
    Name_list = []
    extraversion_list = []
    sensing_list = []
    thinking_list = []
    judging_list = []
    academic_list = []
    
    for student in studygroup:
        Name_list.append(student.Name)
        extraversion_list.append(student.ExScore)
        sensing_list.append(student.SeScore) 
        thinking_list.append(student.ThScore)
        judging_list.append(student.JuScore)
        academic_list.append(student.Academic_Skill)

    data = pd.DataFrame({'Name': Name_list, 
                        'type': student_list, 
                        'E/I': extraversion_list, 
                        'S/N': sensing_list,
                        'T/F': thinking_list,
                        'J/P': judging_list, 
                        'Academic': academic_list})
    
    return data
    

#### Create Study Group

In [5166]:
## Defining the Students ##
class Student():
    def __init__(self, Name, Ex, Se, Th, Ju):
        self.Name  = Name

        ## Personality Traits ## 
        self.Ex = Ex #Extraversion vs Introversion dimension
        self.Se = Se #Sensing vs Intuition dimension
        self.Th = Th #Thinking vs Feeling dimension
        self.Ju = Ju #Judging vs Perceiving dimension

        ## Personality Scores calculated with the personality() function##
        self.ExScore = 0
        self.SeScore = 0
        self.ThScore = 0
        self.JuScore = 0
        
        self.Scores = [] #list of all personality scores

        ## Academic Skills ##
        self.Academic_Skill = 0

        ## Own Solution ##
        self.Ind_Solution = []

def personality(student):
    # Extraversion vs. Introversion
    if student.Ex == "E":
        student.ExScore = max_vals[0]
        del max_vals[0]
    else:
        student.ExScore = min_vals[0]
        del min_vals[0]
    
    # Sensing vs. Intuition
    if student.Se == "S":
        student.SeScore = max_vals[0]
        del max_vals[0]
    else:
        student.SeScore = min_vals[0]
        del min_vals[0]
    
    # Thinking vs. Feeling
    if student.Th == "T":
        student.ThScore = max_vals[0]
        del max_vals[0]
    else:
        student.ThScore = min_vals[0]
        del min_vals[0]

    # Judging vs. Perceiving
    if student.Ju == "J":
        student.JuScore = max_vals[0]
        del max_vals[0]
    else:
        student.JuScore = min_vals[0]
        del min_vals[0]
    
    student.Scores = [student.ExScore, student.SeScore, student.ThScore, student.JuScore]

def skills(student):
    student.Academic_Skill = (1-student.SeScore)*0.40 + student.ThScore*0.20 + (1-student.JuScore)*0.40

In [5167]:
def StudyGroup(student_list):
    '''
    Create a study group of students
    '''
    studygroup = []
    names = ["Alfa", "Bravo", "Charlie", "Delta"]

    for i in range(len(student_list)):
        student = Student(names[i], student_list[i][0], student_list[i][1], student_list[i][2], student_list[i][3])
        personality(student)
        skills(student)
        studygroup.append(student)
    
    return data_collect(studygroup, student_list), studygroup

## Creating the ABM

### Generating True Solution

In [5168]:
def true_solution_generator(n_part_exercises, range_elements):
    '''
    create a list of random numbers that will serve as the true solution that the agents need to find 
    '''
    true_solution = []
    for i in range(n_part_exercises):
        true_solution.append(random.randint(range_elements[0], range_elements[1]))

    return true_solution, range_elements

### Generating Individual Solutions

In [5169]:
def individual_solutions_generator(studygroup, true_solution, range_elements):
    '''
    #function to calculate the individual solutions of the agents given a study group dataframe and a true solution
    '''
    
    all_solutions = []

    for student in studygroup[1]: # loop for each individual
        Ind_Solution_lst = []
        
        for i in range(len(true_solution)): # loop for each part-exercise
            coin_toss = np.random.binomial(1, (student.Academic_Skill*0.7) + 0.3, 1)[0] # biased-coin flip
            if coin_toss == 1:
                Ind_Solution_lst.append(true_solution[i])
            else:
                Ind_Solution_lst.append(random.randint(range_elements[0], range_elements[1]))
        
        all_solutions.append(Ind_Solution_lst)
        student.Ind_Solution = Ind_Solution_lst
    
    return all_solutions

## Collaborative Problem Solving

##### Workflow notes
1. Who presents their solution e.g. agent A presents their solution to agent B, C, D -> THE PROPOSED SOLUTION
    -> Based on Extraversion score (Highest extraversion score is the most likely to present their solution)
2. According to an Agreeableness score (Social score for now) of the other agents (and maybe a Trustworthiness score of agent proposing), agents will update their solution
3. The solutions of the agents will be checked, if all they agree, this is their final solution. If not, the process will be repeated for a max of X ticks. If the groups do not converge, an accuracy score will still be calculated. 

In [5170]:
def collaborative_solution(studygroup, max_ticks):
    max_ticks = max_ticks #turns in the simulation
    n_ticks = 0
    
    # extracting all 4 students
    Alfa = studygroup[1][0]
    Bravo = studygroup[1][1]
    Charlie = studygroup[1][2]
    Delta = studygroup[1][3]

    student_list = [Alfa, Bravo, Charlie, Delta]
    individual_solutions = [Alfa.Ind_Solution, Bravo.Ind_Solution, Charlie.Ind_Solution, Delta.Ind_Solution]
    consensus = False

    while (n_ticks != max_ticks):
        # Starting a Round
        presenter_name = random.choices([Alfa.Name, Bravo.Name, Charlie.Name, Delta.Name], weights = [Alfa.ExScore, Bravo.ExScore, Charlie.ExScore, Delta.ExScore], k = 1)[0] # selecting the presenter of the round based on weighted random draw from extraversion scores
        
        Proposed_Solution = eval(presenter_name).Ind_Solution
        #print(n_ticks, presenter_name, Proposed_Solution)

        for student in student_list:
            if student == eval(presenter_name):
                pass
            for i in range(len(Proposed_Solution)): # looping through all part-exercises and evaluating against proposed solution.
                coin_toss = np.random.binomial(1, ((1-student.ThScore)*0.25 + eval(presenter_name).Academic_Skill*0.25), 1)[0] #high social skill has a greater chance of accepting the proposal and if the proposer has higher academic skill.
                if coin_toss == 1:
                    student.Ind_Solution[i] = Proposed_Solution[i]
                else:
                    student.Ind_Solution[i] = student.Ind_Solution[i]

        n_ticks += 1 #adding a tick to the simulation

        if all(x==individual_solutions[0] for x in individual_solutions): # initializes a break of while-loop if consensus has been reached (i.e. all solutions are the same)
            consensus = True
            break



    #print("Number of iterations: " + str(n_ticks))
    return individual_solutions[0], n_ticks, consensus # returns their collective solution


In [5171]:
def solution_evaluator(proposed_solution, true_solution):
    correct_part_exercise = 0
    wrong_part_exercise = 0

    for i in range(len(true_solution)):
        if proposed_solution[i] == true_solution[i]:
            correct_part_exercise += 1
        else:
            wrong_part_exercise += 1

    return correct_part_exercise / (correct_part_exercise + wrong_part_exercise)


# TEST RUN

In [5172]:
# Selecting Personality Types
student_list_test = ["INTP", "INTP", "INTP", "INTP"]

In [5173]:
# Creating the Study Group
studygroup1 = StudyGroup(student_list_test)

In [5174]:
# Seeing their Generated Personality Scores
print(studygroup1[0])

      Name  type       E/I       S/N       T/F       J/P  Academic
0     Alfa  INTP  0.458662  0.341239  0.776189  0.349198  0.679063
1    Bravo  INTP  0.273450  0.225007  0.650275  0.126414  0.789487
2  Charlie  INTP  0.377716  0.462145  0.670139  0.249398  0.649411
3    Delta  INTP  0.369908  0.417320  0.596576  0.237704  0.657306


In [5175]:
# Generating a True Solution
true_solution_test = true_solution_generator(10, [1, 9])

In [5176]:
# Displaying the True Solution
true_solution_test[0]

[6, 3, 4, 3, 3, 2, 9, 4, 5, 8]

In [5177]:
# Generating Individual Solutions
all_solutions = individual_solutions_generator(studygroup1, true_solution_test[0], true_solution_test[1])
all_solutions

[[6, 3, 4, 3, 3, 2, 9, 4, 5, 8],
 [7, 3, 4, 3, 4, 2, 9, 4, 8, 8],
 [6, 1, 4, 3, 9, 2, 9, 9, 5, 8],
 [6, 3, 4, 3, 3, 2, 9, 4, 5, 8]]

In [5178]:
# Printing Individual Solutions
for student in studygroup1[1]:
    print(student.Name, student.Ind_Solution)

Alfa [6, 3, 4, 3, 3, 2, 9, 4, 5, 8]
Bravo [7, 3, 4, 3, 4, 2, 9, 4, 8, 8]
Charlie [6, 1, 4, 3, 9, 2, 9, 9, 5, 8]
Delta [6, 3, 4, 3, 3, 2, 9, 4, 5, 8]


In [5179]:
# Printing Indivudal Accuracies
for student in studygroup1[1]:
    print(student.Name, solution_evaluator(student.Ind_Solution, true_solution_test[0]))

Alfa 1.0
Bravo 0.7
Charlie 0.7
Delta 1.0


In [5180]:
# Creating Group Solution
group_solution = collaborative_solution(studygroup1, 100)

In [5181]:
# Printing Group Accuracy
accuracy = solution_evaluator(group_solution[0], true_solution_test[0])
print(accuracy)

0.9


## ABM Model

In [5182]:
def exercise_run(student_list: list, max_ticks: int, n_part_exercises: int, range_solution: list, n_simulations: int):
    '''
    function to run one simulation of a study group completing the exercise (from individual to group solution)
    '''

    ## defining the lists that will accumulate the average accuracies, consensus and amount of ticks ## 
    group_accuracy_list = []
    n_tick_list = []
    consensus_list = []

    alfa_accuracy_list = []
    beta_accuracy_list = []
    charlie_accuracy_list = []
    delta_accuracy_list = []


    for i in range(n_simulations):
        ## giving the study group personality values and skills ##
        studygroup = StudyGroup(student_list)

        ## generating the true solution ## 
        true_solution = true_solution_generator(n_part_exercises, range_solution)

        ## generating individual solutions ##
        all_solutions = individual_solutions_generator(studygroup, true_solution[0], true_solution[1])

        ## evaluating own solutions ##        
        alfa_accuracy = solution_evaluator(all_solutions[0], true_solution[0])
        beta_accuracy = solution_evaluator(all_solutions[1], true_solution[0])
        charlie_accuracy = solution_evaluator(all_solutions[2], true_solution[0])
        delta_accuracy = solution_evaluator(all_solutions[3], true_solution[0])
        
        alfa_accuracy_list.append(alfa_accuracy)
        beta_accuracy_list.append(beta_accuracy)
        charlie_accuracy_list.append(charlie_accuracy)
        delta_accuracy_list.append(delta_accuracy)

        ## generating group solution ##
        group_solution = collaborative_solution(studygroup, max_ticks)
        n_tick_list.append(group_solution[1]) # appending the number of ticks
        consensus_list.append(group_solution[2]) #appending consensus values

        ## calculating the accuracy of the group solution ##
        group_accuracy = solution_evaluator(group_solution[0], true_solution[0])
        group_accuracy_list.append(group_accuracy) #appending the group accuracies

    ## calculating the average accuracy and ticks ##
    # group level #
    avg_group_accuracy = np.mean(group_accuracy_list)
    std_group_accuracy = np.std(group_accuracy_list)
    
    avg_group_n_tick = np.mean(n_tick_list)
    std_group_n_tick = np.std(n_tick_list)
    
    avg_group_consensus = np.mean(consensus_list)
    std_group_consensus = np.std(consensus_list)
    
    # individual level #
    avg_alfa_accuracy = np.mean(alfa_accuracy_list)
    std_alfa_accuracy = np.std(alfa_accuracy_list)

    avg_beta_accuracy = np.mean(beta_accuracy_list)
    std_beta_accuracy = np.std(beta_accuracy_list)

    avg_charlie_accuracy = np.mean(charlie_accuracy_list)
    std_charlie_accuracy = np.std(charlie_accuracy_list)

    avg_delta_accuracy = np.mean(delta_accuracy_list)
    std_delta_accuracy = np.std(delta_accuracy_list)
    
    ## dictionaries ## 
    exercise_run_dict_group = {
    'alfa': student_list[0], 
    'beta':student_list[1], 
    'charlie':student_list[2], 
    'delta':student_list[3], 
    'avg_accuracy': avg_group_accuracy, 
    'std_accuracy': std_group_accuracy, 
    'avg_n_tick': avg_group_n_tick, 
    'std_n_tick': std_group_n_tick, 
    'avg_consensus': avg_group_consensus, 
    'std_consensus': std_group_consensus}


    exercise_run_list_ind = list(zip(student_list, 
                                    [student_list, student_list, student_list, student_list], 
                                    [avg_alfa_accuracy, avg_beta_accuracy, avg_charlie_accuracy, avg_delta_accuracy],
                                    [std_alfa_accuracy, std_beta_accuracy, std_charlie_accuracy, std_delta_accuracy]))
    
    return exercise_run_dict_group, exercise_run_list_ind

In [5183]:
def abm_model(condition, max_ticks: int, n_part_exercises: int, range_solution: list, n_simulations: int):
    condition_group_df = pd.DataFrame(columns = ['alfa', 'beta', 'charlie', 'delta', 'avg_accuracy', 'std_accuracy', 'avg_n_tick', 'std_n_tick', 'avg_consensus', 'std_consensus'])
    condition_ind_df = pd.DataFrame(columns=['type', 'studygroup', 'avg_accuracy', 'std_accuracy']) #'avg_accuracy', 'contribution', 'n_presentations'])
        
    for i in (range(len(condition.index))):
        student_list = list(condition.iloc[i][1:5])
        
        ## Getting data frames for the group and individual levels ##
        exercise_run_dict_group, exercise_run_list_ind = exercise_run(student_list, max_ticks, n_part_exercises, range_solution, n_simulations) #creating dictionary
        
        exercise_run_group_df = pd.DataFrame(exercise_run_dict_group, index = [i]) #creating dataframe
        condition_group_df = pd.concat([condition_group_df, exercise_run_group_df], ignore_index = True) #concatenating dataframes
        
        exercise_run_ind_df = pd.DataFrame(exercise_run_list_ind, columns=['type', 'studygroup', 'avg_accuracy', 'std_accuracy']) #'avg_accuracy', 'contribution', 'n_presentations'])
        condition_ind_df = pd.concat([condition_ind_df, exercise_run_ind_df], ignore_index = True)

    return condition_ind_df


In [5184]:
abm_model(condition = homogenous, 
        max_ticks = 30, 
        n_part_exercises = 10, 
        range_solution = [1, 9], 
        n_simulations = 10
        )

,type,studygroup,avg_accuracy,std_accuracy
0,ESFJ,"[ESFJ, ESFJ, ESFJ, ESFJ]",0.58,0.116619
1,ESFJ,"[ESFJ, ESFJ, ESFJ, ESFJ]",0.55,0.143178
2,ESFJ,"[ESFJ, ESFJ, ESFJ, ESFJ]",0.61,0.083066
3,ESFJ,"[ESFJ, ESFJ, ESFJ, ESFJ]",0.59,0.181384
4,ESTJ,"[ESTJ, ESTJ, ESTJ, ESTJ]",0.70,0.154919
...,...,...,...,...
59,ISFP,"[ISFP, ISFP, ISFP, ISFP]",0.62,0.146969
60,ISTP,"[ISTP, ISTP, ISTP, ISTP]",0.71,0.113578
61,ISTP,"[ISTP, ISTP, ISTP, ISTP]",0.67,0.141774
62,ISTP,"[ISTP, ISTP, ISTP, ISTP]",0.66,0.135647


In [5185]:
abm_model(condition = fiftyfifty, 
        max_ticks = 30, 
        n_part_exercises = 10, 
        range_solution = [1, 9], 
        n_simulations = 10
        )

,type,studygroup,avg_accuracy,std_accuracy
0,ESFJ,"[ESFJ, ESFJ, INTP, INTP]",0.62,0.116619
1,ESFJ,"[ESFJ, ESFJ, INTP, INTP]",0.60,0.109545
2,INTP,"[ESFJ, ESFJ, INTP, INTP]",0.75,0.102470
3,INTP,"[ESFJ, ESFJ, INTP, INTP]",0.75,0.120416
4,ENTP,"[ENTP, ENTP, ISFJ, ISFJ]",0.74,0.135647
...,...,...,...,...
59,ISFJ,"[ESTP, ESTP, ISFJ, ISFJ]",0.56,0.111355
60,ENTJ,"[ENTJ, ENTJ, ISTJ, ISTJ]",0.78,0.107703
61,ENTJ,"[ENTJ, ENTJ, ISTJ, ISTJ]",0.70,0.126491
62,ISTJ,"[ENTJ, ENTJ, ISTJ, ISTJ]",0.60,0.141421


In [5186]:
abm_model(condition = heterogenous, 
        max_ticks = 30, 
        n_part_exercises = 10, 
        range_solution = [1, 9], 
        n_simulations = 10
        )

,type,studygroup,avg_accuracy,std_accuracy
0,ESFJ,"[ESFJ, ESTJ, ENFP, INFJ]",0.49,0.130000
1,ESTJ,"[ESFJ, ESTJ, ENFP, INFJ]",0.69,0.130000
2,ENFP,"[ESFJ, ESTJ, ENFP, INFJ]",0.67,0.064031
3,INFJ,"[ESFJ, ESTJ, ENFP, INFJ]",0.78,0.097980
4,ISFJ,"[ISFJ, ESTP, ISFP, ENFJ]",0.64,0.190788
...,...,...,...,...
59,INFJ,"[ESTJ, ISTJ, ENFJ, INFJ]",0.65,0.120416
60,ENTJ,"[ENTJ, INTJ, ESTP, ISTP]",0.72,0.107703
61,INTJ,"[ENTJ, INTJ, ESTP, ISTP]",0.75,0.120416
62,ESTP,"[ENTJ, INTJ, ESTP, ISTP]",0.74,0.101980
